# import libraries:

In [ ]:
import numpy as np
import sep
# additional setup for reading the test image and displaying plots
from astropy.io import fits
#import fitsio
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

# Reading in the image as well as displaying it

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
data = fits.getdata(fname)
data = data.astype(np.float32) #need to reduce the size of the data or else it will cause an error 
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower') #showing the image
plt.colorbar();
plt.savefig('originalimage.png')

# Process of removing the background

In [ ]:
bkg = sep.Background(data) 

# getting the global mean of the background:

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

# stores background into 2d array and prints it: 

In [ ]:
bkg_image = bkg.back()
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('backgroundimage.png')

# getting the background noise:

In [ ]:
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('noiseimage.png')

# subtracting the background noise from

In [ ]:
data_sub = data - bkg

# start trying to detect objects

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms) #threshold = 1.5
#Now we will print out the number of objects detected
print("The number of sources found: ", len(objects))
print(type(objects))

# Now we will circle all of the objects which were detected

In [ ]:
# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('circledimage.png')

# Make a histogram: 

In [ ]:
#getting the flux data to start off 
new = objects['flux']
#log the flux result because some points are really big
new = np.log10(new)

In [ ]:
histogram = plt.hist(new, bins='auto')
plt.xlabel('value')
plt.ylabel('occurances')
plt.title('Histogram of fluxes')

In [ ]:
#q = acszpt.Query(date='2002-12-06', detector='WFC', filt='F105W')
#filter_zpt = q.fetch()
m_ab = -2.5 * np.log10(objects['flux']) - 48.60
histogram = plt.hist(m_ab, bins='auto')
plt.xlabel('value')
plt.ylabel('occurances')
plt.title('Histogram of ab magnitude')